In [68]:
#Importing needed packages
!pip install geopy

!pip install folium

import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

import requests
import lxml.html as lh

print('Libraries imported.')

Libraries imported.


# Part 1: Creating a Dataframe From a Wikipedia Page

In [83]:
#importing the table from a html webpage
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [84]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
dfs[0].head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [85]:
#dropping NaN rows
dfs=dfs[0].dropna()

In [86]:
#merging rows with the same postal code
dfs=dfs.groupby(['Postal code','Borough'], sort = False).agg(lambda x: ','.join(x))
dfs

,,Neighborhood
Postal code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Malvern / Rouge
M3B,North York,Don Mills
M4B,East York,Parkview Hill / Woodbine Gardens


In [87]:
dfs.shape

(103, 1)

# Part 2: Merging the Neighborhood Table with Latitude and Longitude Information

In [88]:
#importing the csv file with the latitude and longitude information
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [89]:
#renaming the Postal Code column to be the same name as the column in the Neighborhood table
df_geo=df_geo.rename(columns={"Postal Code": "Postal code"})

In [90]:
#merging the 2 tables
dfs=pd.merge(dfs, df_geo, on='Postal code')

In [92]:
dfs

,Postal code,Neighborhood,Latitude,Longitude
0,M3A,Parkwoods,43.753259,-79.329656
1,M4A,Victoria Village,43.725882,-79.315572
2,M5A,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Islington Avenue,43.667856,-79.532242
6,M1B,Malvern / Rouge,43.806686,-79.194353
7,M3B,Don Mills,43.745906,-79.352188
8,M4B,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,"Garden District, Ryerson",43.657162,-79.378937


# Part 3: Exploring the Data

In [69]:
#finding the coordinates for Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [95]:
#creating a map with the neighborhoods on it as markers in order to visualize the layout of the neighborhoods
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood in zip(dfs['Latitude'], dfs['Longitude'], dfs['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Observations: As we can see from the map above, most neighborhoods are fairly uniformly spreadout with the expection of the downtown core area. The downtown core area is very densely pacted with neighborhoods